This notebook requires paramiko, sshtunnel, and pymysql


"pip install paramiko"
"pip install sshtunnel"
"pip install pymysql"

Paramiko is optional and is only necessary if you want to read from the RSA private key file to get ssh access.

In [1]:
import numpy as np
import datetime as dt
import os
import time
import paramiko
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import pymysql

In [17]:
"""Connect to the MYSQL database hosted on OCF's server and run queries!"""
sql_hostname = 'mysql'
sql_username = 'ankurmahesh'
with open("sql_password") as f:
    sql_password = f.read()

with open("ssh_password") as f:
    ssh_password = f.read()

sql_main_database = 'ankurmahesh'
sql_port = 3306
ssh_host = 'ssh.ocf.berkeley.edu'
ssh_user = 'ankurmahesh'
ssh_port = 22
#mypkey = paramiko.RSAKey.from_private_key_file(pkeyfilepath)

date = "01-01-01"
temperature1 = 75.0
temperature2 = 75.7
pressure = 1.0
humidity = 5.0

with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_user,
    ssh_password=ssh_password,
    #ssh_pkey=mypkey,
    remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
        passwd=sql_password, db=sql_main_database,
        port=tunnel.local_bind_port, autocommit=True)
    #sample read query
    query = "SELECT temperature1 from weatherdata;"
    data = pd.read_sql_query(query, conn)
    
    #sample insert into database
    insert_command = "INSERT INTO weatherdata VALUES({},{},{},{},{});".format(date, temperature1, \
                                                                    temperature2, pressure, humidity)
    cur = conn.cursor()
    cur.execute(insert_command)
    conn.close() 

In [18]:
data

,temperature1
0,1.0
1,75.0


In [12]:
insert_command

'INSERT INTO weatherdata VALUES(%s,%f,%f,%f,%f);'